# Figure 1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pylab as plt
#import delfi.distribution as dd
import time
from copy import deepcopy
import sys
sys.path.append("../../../setup")
sys.path.append("../../../simulator")
sys.path.append("../../../inference")
sys.path.append("../../../utils")
sys.path.append("../model/")
sys.path.append('../../')

from common import col, svg, plot_pdf, samples_nd, get_labels_8pt
from find_pyloric import merge_samples, params_are_bounded

import netio
import viz
import importlib
import train_utils as tu

import matplotlib as mpl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
PANEL_G1 = '../svg/panel_g1.svg'
PANEL_G2 = '../svg/panel_g2.svg'
PANEL_B1 = '../svg/panel_b1.svg'
PANEL_B2 = '../svg/panel_b2.svg'
PANEL_B3 = '../svg/panel_b3.svg'
PANEL_B4 = '../svg/panel_b4.svg'

### Load samples

In [4]:
params = netio.load_setup('collect_31D')

In [5]:
outfile = '../../results/11deg_post_pred/11_deg_post_pred.npz'
data_n = np.load(outfile)
sample_seeds = data_n["seeds"]
sample_params = data_n["params"]
sample_stats = data_n["stats"]
print('There are', len(sample_seeds), 'datapoints in this dataset')

stats_mean = np.mean(sample_stats, axis=0)
stats_std  = np.std(sample_stats, axis=0)

There are 1124507 datapoints in this dataset


### Load network

In [6]:
prior = netio.create_prior(params, log=True)
dimensions = np.sum(params.use_membrane) + 7
lims = np.asarray([-np.sqrt(3)*np.ones(dimensions), np.sqrt(3)*np.ones(dimensions)]).T

In [7]:
params_mean = prior.mean
params_std = prior.std

In [8]:
sample_params_norm = (sample_params - params_mean) / params_std

In [9]:
from find_pyloric import merge_samples, params_are_bounded

#prior_normalized = dd.Uniform(-np.sqrt(3)*np.ones(dimensions), np.sqrt(3)*np.ones(dimensions), seed=params.seed)

# Panel B: experimental data

In [10]:
npz = np.load('../../results/experimental_data/trace_data_845_082_0044.npz')
t = npz['t']
PD_spikes = npz['PD_spikes']
LP_spikes = npz['LP_spikes']
PY_spikes = npz['PY_spikes']
pdn = npz['pdn']
lpn = npz['lpn']
pyn = npz['pyn']

In [11]:
start_index = 219500 + 2100
end_index   = 246500 + 2100  # 32000
height_offset = 200
shown_t = t[end_index] - t[start_index]
time_len = shown_t / 0.025 * 1000
dt = t[1] - t[0]

In [12]:
import matplotlib.patches as mp

# Panel C: posterior

In [13]:
from decimal import Decimal
all_labels = []
for dim_i in range(31):
    if dim_i > len(params_mean) - 7.5: # synapses
        if dim_i == 24: all_labels.append([r'$\mathdefault{0.01}$ ', r'$\mathdefault{10000}\;\;\;\;$  '])
        else: all_labels.append([r'$\;\;\mathdefault{0.01}$', r'$\mathdefault{1000}\;\;\;\;$ '])
    else: # membrane conductances
        num_after_digits = -int(np.log10(lims[dim_i, 1] * params_std[dim_i] + params_mean[dim_i]))
        if num_after_digits > 2:
            num_after_digits=2
        labels = [round(Decimal((lims[dim_i, num_tmp] * params_std[dim_i] + params_mean[dim_i]) / 0.628e-3), num_after_digits)
                  for num_tmp in range(2)]
        new_labels = []
        counter=0
        for l in labels:
            if counter == 0:
                new_labels.append(r'$\mathdefault{'+str(l)+'}$')
            else:
                new_labels.append(r'$\mathdefault{'+str(l)+'}\;\;\;$ ')
            counter+=1
        all_labels.append(new_labels)

# Panel D: posterior samples

In [35]:
params['noise_fact'] = 0.0

In [36]:
pyloric_sim = netio.create_simulators(params, save_all_energies=True)[0]
summ_stats = netio.create_summstats(params)

('seed in cHH', None)


In [37]:
seeds = [8, 8, 8, 8, 8]
all_traces = []
for parameter_index in range(2):
    all_traces.append(pyloric_sim.gen_single(deepcopy(sample_params[parameter_index]),
                      seed_sim=True,
                      to_seed=seeds[parameter_index])
                     )

In [39]:
#labels_ = viz.get_labels_8pt(params)
offset = 39000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[0],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='g', time_len=int(time_len/5),
    )
    
    plt.savefig(PANEL_G1, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

In [40]:
#labels_ = viz.get_labels_8pt(params)
offset = 60000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[1],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='g', time_len=int(time_len/6),
    )
    
    plt.savefig(PANEL_G2, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

### Bad parameter sets

In [41]:
sample = prior.gen(1)
#np.savez('prior_sample_1.npz', sample=sample)

In [42]:
sample = np.load('prior_sample_1.npz')['sample']

In [43]:
seeds = [8, 8, 8, 8, 8]
all_traces = []
for parameter_index in range(1):
    all_traces.append(pyloric_sim.gen_single(deepcopy(sample[0]),
                      seed_sim=True,
                      to_seed=seeds[parameter_index]))

In [44]:
#labels_ = viz.get_labels_8pt(params)
offset = 58000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[0],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='#e31a1c', time_len=int(time_len/3),
    )
    
    plt.savefig(PANEL_B1, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

In [45]:
sample = prior.gen(1)
#np.savez('prior_sample_2.npz', sample=sample)

In [46]:
sample = np.load('prior_sample_2.npz')['sample']

In [47]:
seeds = [8, 8, 8, 8, 8]
all_traces = []
for parameter_index in range(1):
    all_traces.append(pyloric_sim.gen_single(deepcopy(sample[0]),
                      seed_sim=True,
                      to_seed=seeds[parameter_index]))

In [48]:
#labels_ = viz.get_labels_8pt(params)
offset = 60000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[0],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='#e31a1c', time_len=int(time_len/4),
    )
    
    plt.savefig(PANEL_B2, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

In [49]:
sample = prior.gen(1)
#np.savez('prior_sample_3.npz', sample=sample)

In [50]:
sample = np.load('prior_sample_3.npz')['sample']

In [51]:
seeds = [8, 8, 8, 8, 8]
all_traces = []
for parameter_index in range(1):
    all_traces.append(pyloric_sim.gen_single(deepcopy(sample[0]),
                      seed_sim=True,
                      to_seed=seeds[parameter_index]))

In [52]:
#labels_ = viz.get_labels_8pt(params)
offset = 60000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[0],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='#e31a1c', time_len=int(time_len/4),
    )
    
    plt.savefig(PANEL_B3, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

In [53]:
sample = prior.gen(1)
#np.savez('prior_sample_4.npz', sample=sample)

In [54]:
sample = np.load('prior_sample_4.npz')['sample']

In [55]:
seeds = [8, 8, 8, 8, 8]
all_traces = []
for parameter_index in range(1):
    all_traces.append(pyloric_sim.gen_single(deepcopy(sample[0]),
                      seed_sim=True,
                      to_seed=seeds[parameter_index]))

In [56]:
#labels_ = viz.get_labels_8pt(params)
offset = 60000

fig, ax = plt.subplots(1,1, figsize=(3,0.3))

with mpl.rc_context(fname='../../.matplotlibrc'):

    fig = viz.vis_ABPD_plain(
        all_traces[0],
        t=pyloric_sim.t,
        axV = ax,
        offset=offset,
        col='#e31a1c', time_len=int(time_len/4),
    )
    
    plt.savefig(PANEL_B4, facecolor='None', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()